In [132]:
# Important libraries
from dotenv import load_dotenv
import os
import pandas as pd
from langchain_groq import ChatGroq

# Langchain 
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from transformers.integrations import integration_utils

# # Import embedding model
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate

# Vector store...
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

import warnings         # Ignore all warnings
warnings.filterwarnings("ignore")

In [133]:
# Load in the dataset
context = pd.read_csv("./drugs_side_effects_drugs_com.csv")
context.head(2)

,drug_name,medical_condition,side_effects,generic_name,drug_classes,brand_names,activity,rx_otc,pregnancy_category,csa,alcohol,related_drugs,medical_condition_description,rating,no_of_reviews,drug_link,medical_condition_url
0,doxycycline,Acne,"(hives, difficult breathing, swelling in your ...",doxycycline,"Miscellaneous antimalarials, Tetracyclines","Acticlate, Adoxa CK, Adoxa Pak, Adoxa TT, Alod...",87%,Rx,D,N,X,amoxicillin: https://www.drugs.com/amoxicillin...,Acne Other names: Acne Vulgaris; Blackheads; B...,6.8,760.0,https://www.drugs.com/doxycycline.html,https://www.drugs.com/condition/acne.html
1,spironolactone,Acne,hives ; difficulty breathing; swelling of your...,spironolactone,"Aldosterone receptor antagonists, Potassium-sp...","Aldactone, CaroSpir",82%,Rx,C,N,X,amlodipine: https://www.drugs.com/amlodipine.h...,Acne Other names: Acne Vulgaris; Blackheads; B...,7.2,449.0,https://www.drugs.com/spironolactone.html,https://www.drugs.com/condition/acne.html


In [134]:
# Embedding model...
embed_model = HuggingFaceEmbeddings(model_name="mixedbread-ai/mxbai-embed-large-v1")

# Initialize the model
llm_model = ChatGroq(model="llama-3.3-70b-versatile", api_key=os.environ.get("GROQ_API_KEY"))

In [135]:
# Database for saving the documents
vectorstore = Chroma(
    collection_name="medical_dataset_store",
    embedding_function=embed_model,
    persist_directory="./",
)
vectorstore.get().keys()

# Because the vector store is empty... Add your context data.
vectorstore.add_texts(context)

['e44a754d-37fc-4547-9043-2a7841221610',
 'f29f8a40-f13b-4f70-a8c6-027a5586a9eb',
 '1d15754d-6456-4502-b08d-acb3e0de03e8',
 'b5478677-5770-4c1b-9b54-04bc9cb9f61d',
 '6a8c3cd7-52ab-4b90-aa2e-34efe41fdc68',
 'e1c91251-7e8b-4a11-bf0e-e5787c2431e4',
 '8b02b816-d210-4f21-bbbe-516e054fc22f',
 '754c3787-2f1b-40b2-82c9-533df4887988',
 '169752d5-95cc-4950-bbc0-2671cd0fdaa4',
 '9bd556e8-352c-419a-9464-8c7407d298f5',
 'dd79f815-24af-4735-a112-52a48ddca216',
 'da1de8e2-5dfa-4639-9b6c-3c8182a60981',
 'ef1bb6b5-72c5-4332-83a0-329537e23ccc',
 '42b07d45-defe-4969-81d1-4a9c3d4d4cca',
 '2a34d8a2-75fe-4f94-8949-58c09d7e4a00',
 '05bcd9fb-a826-4511-9481-9d1a841fabac',
 '9d4941d8-95c2-4d94-ae92-22be64dca94a']

In [136]:
# Load the retriever for fetching the data..
retriever = vectorstore.as_retriever()

In [137]:
template = """
You are a specialized medical assistant with expertise in pharmacology, evidence-based medicine, and medical diagnostics. Your role is to analyze queries, apply logical reasoning, and deliver accurate, reliable, and well-structured responses using the provided "{context}". Follow these guidelines:

### Context:
{context}

### Question:
{question}

### Answer:

1. **Understand the Scope**:
   - Ensure you get correctly the first question.
   - General questions: Provide a broad response.
   - Specific questions: Provide detailed information.
   - Unclear questions: Ask for clarification.

2. **Critical Thinking**:
   - Analyze the question and cross-check with the context.
   - Avoid over-specificity unless explicitly requested.

3. **Accuracy and Alignment**:
   - Align answers with the question's intent.
   - Avoid vague responses; confirm facts or state limitations.

4. **Professional Tone and Formatting**:
   - Use non-technical language for general audiences.
   - Structure responses with bullet points or clear paragraphs.
   - Summarize long responses for clarity.

5. **Error Handling**:
   - If unsure, respond honestly: "Please consult a healthcare professional."
   - Request clarification for unclear questions.

6. **Advanced Reasoning**:
   - Side Effects: Categorize as common or severe, and mention when to seek medical attention.
   - Drug Interactions: Explain risks and suggest precautions.
   - Medical Conditions: Provide causes, symptoms, risks, treatments, and prevention tips.

7. **Handling Irrelevant Questions**:
   - If unrelated to the context or outside your scope: "Please consult a healthcare professional."
   - Correct any missed or irrelevant responses in the next answer.


"""
rag_prompt = PromptTemplate.from_template(template)

In [138]:
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm_model
    | StrOutputParser()
)

In [145]:
import gradio as gr

def rag_memory_stream(message, history):
    partial_text = ""
    for new_text in rag_chain.stream(message):
        partial_text += new_text
        yield partial_text

examples = [
    "What is a drug ?", 
    "What are the side effects of lisinopril?"
]

description = "Real-Time AI-Powered Medical Assistant: Drug Side Effect Queries Chatbot"


title = "AI-Powered Medical Chatbot :) Try me!"
demo = gr.ChatInterface(fn=rag_memory_stream,
                        type="messages",
                        title=title,
                        description=description,
                        fill_height=True,
                        examples=examples,
                        theme="glass",
)

# Launch the application and make it sharable
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7905
* Running on public URL: https://6e024a6915dc748c3d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [151]:
import gradio as gr

def rag_memory_stream(message, history):
    try:
        # Combine the chat history with the new message
        full_query = "\n".join([f"User: {msg[0]}\nBot: {msg[1]}" for msg in history]) + f"\nUser: {message}"
        
        # Stream the response
        partial_text = ""
        for new_text in rag_chain.stream(full_query):
            partial_text += new_text
            yield partial_text
        
        # Add a feedback button after the response
        yield partial_text + "\n\nWas this response helpful? [👍] [👎]"
    except Exception as e:
        yield f"An error occurred: {str(e)}. Please try again later."

examples = [
    "What is a drug ?", 
    "What are the side effects of lisinopril?"
]

description = "Real-Time AI-Powered Medical Assistant: Drug Side Effect Queries Chatbot"
title = "AI-Powered Medical Chatbot :) Try me!"

demo = gr.ChatInterface(
    fn=rag_memory_stream,
    type="messages",
    title=title,
    description=description,
    fill_height=True,
    examples=examples,
    flagging_options=True,
    theme=gr.themes.Glass(
        primary_hue="blue",
        secondary_hue="orange",
        neutral_hue="gray"
    ),
    # system_prompt="Welcome! I'm your AI-powered medical assistant. Ask me anything about drugs, side effects, or medical conditions.",
    # clear_btn="Clear Chat",
    # retry_btn="Retry",
    # undo_btn="Undo"
)

# Launch the application and make it sharable
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7907
* Running on public URL: https://5c751c57946475b0ce.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [143]:
import gradio as gr

# def rag_memory_stream(text):
#     partial_text = ""
#     for new_text in rag_chain.stream(text):
#         partial_text += new_text
#         yield partial_text


# title = "Real-Time AI-Powered Medical Assistant: Drug Side Effect Queries Chatbot"
# demo = gr.Interface(
#     title=title,
#     fn=rag_memory_stream,
#     inputs="text",
#     outputs="text",
#     allow_flagging="never",
# )

# demo.launch(share=True)

# Define the streaming function
def rag_memory_stream(message_history, new_user_input):
    # Debug: Print the message history to understand its structure
    print("Message History:", message_history)
    
    # Combine the message history and new user input into a single query
    if message_history:
        full_query = "\n".join([f"User: {msg[0]}\nBot: {msg[1]}" for msg in message_history if len(msg) == 2]) + f"\nUser: {new_user_input}"
    else:
        full_query = f"User: {new_user_input}"
    
    # Stream the response from the rag_chain
    partial_text = ""
    for chunk in rag_chain.stream(full_query):  # Ensure rag_chain supports streaming
        partial_text += chunk
        yield partial_text

# Define the title of the application
title = "Real-Time AI-Powered Medical Assistant: Drug Side Effect Queries Chatbot"

# Create a Gradio ChatInterface
demo = gr.ChatInterface(
    fn=rag_memory_stream,  # Function to process user input
    title=title,           # Title of the interface
    save_history=True,     # Saves chat history if supported
    type="messages"        # Set type to "messages" to enable save_history
)

# Launch the application and make it sharable
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7904
* Running on public URL: https://e432556ca25224a21f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Message History: What is a drug ?
Message History: What is a drug ?
